In [1]:
%pip install -U pandas numpy==1.20 scikit-learn daal Scipy
%pip install tensorflow
%pip install --upgrade --force-install matplotlib


Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.8.0-cp39-cp39-win_amd64.whl (438.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\julie\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



Note: you may need to restart the kernel to use updated packages.



Usage:   
  C:\Users\julie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  C:\Users\julie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  C:\Users\julie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <vcs project url> ...
  C:\Users\julie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <local project path> ...
  C:\Users\julie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install [options] <archive url/path> ...

no such option: --force-install


In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

__version__ = "0.1.0"
__author__ = "Abien Fred Agarap"

import numpy as np
import os
import tensorflow as tf


class NearestNeighbor:
    def __init__(self, train_features, train_labels, sequence_length):
        self.train_features = train_features
        self.train_labels = train_labels

        with tf.name_scope("input"):
            xtr = tf.placeholder(dtype=tf.float32, shape=[None, sequence_length])
            xte = tf.placeholder(dtype=tf.float32, shape=[sequence_length])

        # L1-Norm
        # distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)

        # L2-Norm
        distance = tf.sqrt(tf.reduce_sum(tf.square(xtr - xte), reduction_indices=1))

        prediction = tf.arg_min(distance, 0)

        accuracy = 0.0

        self.xtr = xtr
        self.xte = xte
        self.distance = distance
        self.prediction = prediction
        self.accuracy = accuracy

    def predict(self, test_features, test_labels, result_path):

        train_labels = tf.one_hot(self.train_labels, depth=2, on_value=1, off_value=0)
        test_labels = tf.one_hot(test_labels, depth=2, on_value=1, off_value=0)

        init = tf.global_variables_initializer()

        # Start training
        with tf.Session() as sess:

            # Run the initializer
            sess.run(init)

            y_, y = sess.run([test_labels, train_labels])

            # loop over test data
            for index in range(len(test_features)):

                feed_dict = {
                    self.xtr: self.train_features,
                    self.xte: test_features[index, :],
                }

                nn_index = sess.run(self.prediction, feed_dict=feed_dict)

                print(
                    "Test [{}] Actual Class: {}, Predicted Class : {}".format(
                        index, np.argmax(y_[index]), np.argmax(y[nn_index])
                    )
                )

                self.save_labels(
                    predictions=np.argmax(y[nn_index]),
                    actual=np.argmax(y_[index]),
                    result_path=result_path,
                    step=index,
                    phase="testing",
                )

                if np.argmax(y[nn_index]) == np.argmax(y_[index]):
                    self.accuracy += 1.0 / len(test_features)

        print("Accuracy : {}".format(self.accuracy))

    @staticmethod
    def save_labels(predictions, actual, result_path, step, phase):
        """Saves the actual and predicted labels to a NPY file
        Parameter
        ---------
        predictions : int
          The predicted label.
        actual : int
          The actual label.
        result_path : str
          The path where to save the concatenated actual and predicted labels.
        step : int
          The time step for the predicted and actual labels
        phase : str
          The phase for which the prediction is, i.e. training/validation/testing.
        """

        if not os.path.exists(path=result_path):
            os.mkdir(result_path)

        # Concatenate the predicted and actual labels
        labels = np.array([predictions, actual])

        # save the labels array to NPY file
        """
        np.save(
            file=os.path.join(
                result_path, "{}-nearest_neighbor-{}.npy".format(phase, step)
            ),
            arr=labels,
        )
        """

In [35]:
import scipy.io as sp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

dataset = sp.loadmat('breastw.mat')

features = dataset['X']
labels = dataset['y']

num_features = features.shape[1]

features = StandardScaler().fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, stratify=labels)

model = NearestNeighbor(train_features, train_labels, num_features)

model.predict(test_features, test_labels, result_path="./")

print(model.accuracy)

Test [0] Actual Class: 0, Predicted Class : 0
Test [1] Actual Class: 1, Predicted Class : 1
Test [2] Actual Class: 0, Predicted Class : 0
Test [3] Actual Class: 0, Predicted Class : 0
Test [4] Actual Class: 0, Predicted Class : 0
Test [5] Actual Class: 1, Predicted Class : 1
Test [6] Actual Class: 0, Predicted Class : 0
Test [7] Actual Class: 1, Predicted Class : 1
Test [8] Actual Class: 1, Predicted Class : 1
Test [9] Actual Class: 1, Predicted Class : 1
Test [10] Actual Class: 0, Predicted Class : 0
Test [11] Actual Class: 0, Predicted Class : 0
Test [12] Actual Class: 0, Predicted Class : 0
Test [13] Actual Class: 0, Predicted Class : 0
Test [14] Actual Class: 0, Predicted Class : 0
Test [15] Actual Class: 0, Predicted Class : 0
Test [16] Actual Class: 1, Predicted Class : 1
Test [17] Actual Class: 0, Predicted Class : 0
Test [18] Actual Class: 0, Predicted Class : 0
Test [19] Actual Class: 1, Predicted Class : 1
Test [20] Actual Class: 0, Predicted Class : 0
Test [21] Actual Class: